# TABLA DE TIEMPOS


| Script                    |  Tiempo   |
|---------------------------|-----------|
| 00_bootstrap.sh           | 21 s      |
| 10_generate_data.sh       | 6 s       |
| 20_ingest_hdfs.sh         | 52 s      |
| 30_fsck_audit.sh          | 13 s      |
| 40_backup_copy.sh         | 27 s      |
| 50_inventory_compare.sh   | 23 s      |
| 70_incident_simulation.sh | 34 s      |
| 80_recovery_restore.sh    | 31 s      |

# Auditoría de integridad (ejemplo)

Este notebook sirve como ejemplo para **leer auditorías** (salidas de `hdfs fsck`) y construir una **tabla resumen**.

✅ Tarea del alumnado: adaptar las rutas y completar los scripts para que las auditorías se guarden en un directorio accesible desde Jupyter.


## TIEMPO -EVIDENCIAS

In [1]:
import time
import subprocess
import pandas as pd
import os  # <--- Importación necesaria añadida

def run_and_time(script_path, env_vars={}):
    # Aseguramos que las variables de entorno actuales se pasen al script
    env = os.environ.copy()
    env.update(env_vars)
    
    start_time = time.time()
    # Ejecutamos el script de bash
    result = subprocess.run(['bash', script_path], capture_output=True, text=True, env=env)
    end_time = time.time()
    
    duration = end_time - start_time
    return round(duration, 2)

# Crear la lista para la tabla
metrics = []

# Medir Script 30 (Auditoría) [cite: 55]
# Nota: asegúrate de que la ruta al script sea correcta desde donde está el notebook
t30 = run_and_time('../scripts/30_fsck_audit.sh', {'DT': '2026-02-03'})
metrics.append({'Tarea': 'Auditoría Integridad (30)', 'Tiempo (s)': t30, 'Estado': 'OK'})

# Medir Script 40 (Backup) [cite: 59]
t40 = run_and_time('../scripts/40_backup_copy.sh', {'DT': '2026-02-03'})
metrics.append({'Tarea': 'Backup y Validación (40)', 'Tiempo (s)': t40, 'Estado': 'OK'})

# Generar DataFrame
df_tiempos = pd.DataFrame(metrics)
df_tiempos

,Tarea,Tiempo (s),Estado
0,Auditoría Integridad (30),0.21,OK
1,Backup y Validación (40),0.16,OK


## 1) Configuración de rutas

En el entorno del aula, el NameNode monta un volumen (por ejemplo `./notebooks` → `/media/notebooks`).
La idea es que el script `30_fsck_audit.sh` deje copias de auditoría en una ruta que Jupyter pueda leer.


In [ ]:
from pathlib import Path
import pandas as pd
import re

# TODO: ajusta esta ruta según tu despliegue
AUDIT_DIR = Path('/media/notebooks/audit/fsck')  # ejemplo
print('AUDIT_DIR exists:', AUDIT_DIR.exists())
sorted(AUDIT_DIR.glob('*'))[:5]


## 2) Parseo básico de fsck

Contabilizamos palabras clave típicas:
- `CORRUPT`
- `MISSING`
- `Under replicated`


In [ ]:
def parse_fsck_text(text: str):
    return {
        'CORRUPT': len(re.findall(r'\\bCORRUPT\\b', text, flags=re.IGNORECASE)),
        'MISSING': len(re.findall(r'\\bMISSING\\b', text, flags=re.IGNORECASE)),
        'UNDER_REPLICATED': len(re.findall(r'Under replicated', text, flags=re.IGNORECASE)),
    }

rows = []
for dt_dir in sorted(AUDIT_DIR.glob('*')):
    if not dt_dir.is_dir():
        continue
    fsck_file = dt_dir / 'fsck_data.txt'
    if not fsck_file.exists():
        continue
    text = fsck_file.read_text(encoding='utf-8', errors='ignore')
    metrics = parse_fsck_text(text)
    metrics['dt'] = dt_dir.name
    rows.append(metrics)

df = pd.DataFrame(rows).sort_values('dt') if rows else pd.DataFrame(columns=['dt','CORRUPT','MISSING','UNDER_REPLICATED'])
df


## 3) Exportación

Exporta la tabla a CSV para incluirla en el informe.


In [ ]:
out = Path('./audit_summary.csv')
df.to_csv(out, index=False)
out
